<a href="https://colab.research.google.com/github/Shrey-Viradiya/NVIDIA_DALI_PyTorch-Example/blob/main/NvidiaDALI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sat Feb 27 12:25:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install --extra-index-url https://developer.download.nvidia.com/compute/redist nvidia-dali-cuda100

Looking in indexes: https://pypi.org/simple, https://developer.download.nvidia.com/compute/redist
     |████████████████████████████████| 389.9MB 41kB/s 


In [3]:
import os
from os import listdir
from os.path import isfile, join
import numpy as np
from nvidia.dali.plugin.pytorch import DALIGenericIterator
from nvidia.dali.pipeline import Pipeline
import nvidia.dali.ops as ops
import nvidia.dali.types as types
from nvidia.dali.plugin.pytorch import DALIGenericIterator

# import types
import collections
import pandas as pd
from random import shuffle

In [4]:
!wget -cq https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip \
  && unzip -qq flower_data.zip

In [5]:
# !wget -cq https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip \
#   && unzip -qq flower_data.zip \
#   && mkdir -p ./flower_data/flower_data_flat \
#   && find ./flower_data/train -mindepth 2 -type f -exec mv -t ./flower_data/flower_data_flat -i '{}' +

In [6]:
images_directory = '/content/flower_data/train'
# read names of all image files
# image_files = [f for f in listdir(images_directory) if isfile(join(images_directory, f))]

In [7]:
# # we create a data frame with the image names and dummy labels - label_1, label_2
# labels = []
# for i in range(len(image_files)):
#     x = random.randint(0,4)
#     labels.append(x)
# data = pd.DataFrame(list(zip(image_files, labels)), columns=['image_filename', 'label'])

# processed_data_file = 'flower_dummy_data.ssv'
# data.to_csv(processed_data_file, index=False, header=False, sep=' ')

# print(data.head())
# #	image_filename	label_1	label_2
# # 0	image_05973.jpg	0	    0
# # 1	image_00956.jpg	1	    1
# # 2	image_06047.jpg	2	    2
# # 3	image_07168.jpg	3	    3
# # 4	image_04466.jpg	4	    4

In [8]:
# class ExternalInputIterator(object):
#     def __init__(self, batch_size, data_file, images_directory):
#         self.images_dir = images_directory
#         self.batch_size = batch_size
#         self.data_file = data_file
#         with open(self.data_file, 'r') as f:
#             self.files = [line.rstrip() for line in f if line is not '']

#     def __iter__(self):
#         self.i = 0
#         self.n = len(self.files)
#         shuffle(self.files)
#         return self

#     def __next__(self):

#         if self.i >= self.n:
#             self.__iter__()
#             raise StopIteration

#         batch = []
#         labels = []
#         for _ in range(self.batch_size):
#             # *label reads multiple labels 
#             jpeg_filename, label = self.files[self.i % self.n].split(' ')
#             f = open(images_directory + jpeg_filename, 'rb')
#             batch.append(np.frombuffer(f.read(), dtype = np.uint8))
#             labels.append(np.array(label, dtype = np.uint8))
#             self.i += 1
#         return (batch, labels)

#     next = __next__

In [9]:
# eii = ExternalInputIterator(batch_size=16, 
#                             data_file=processed_data_file, 
#                             images_directory=images_directory)
# iterator = iter(eii)

# class ExternalSourcePipeline(Pipeline):
#     def __init__(self, data_iterator, batch_size, num_threads, device_id):
#         super(ExternalSourcePipeline, self).__init__(batch_size,
#                                       num_threads,
#                                       device_id,
#                                       seed=12)
#         self.data_iterator = data_iterator
#         self.input = ops.ExternalSource()
#         self.input_label = ops.ExternalSource()
#         self.decode = ops.ImageDecoder(device = "mixed", output_type = types.RGB)
#         # resizing is *must* because loaded images maybe of different sizes
#         # and to create GPU tensors we need image arrays to be of same size
#         self.res = ops.Resize(device="gpu", resize_x=224, resize_y=224, interp_type=types.INTERP_TRIANGULAR)

#     def define_graph(self):
#         self.jpegs = self.input()
#         self.labels = self.input_label()
#         images = self.decode(self.jpegs)
#         output = self.res(images)
#         return (output, self.labels)

#     def iter_setup(self):
#         # the external data iterator is consumed here and fed as input to Pipeline
#         images, labels = self.data_iterator.next()
#         self.feed_input(self.jpegs, images)
#         self.feed_input(self.labels, labels)

In [10]:
class HybridPipeline(Pipeline):
    def __init__(self, batch_size, num_threads, device_id, images_directory):
        super(HybridPipeline, self).__init__(batch_size, num_threads, device_id, seed = 12)
        self.input = ops.FileReader(file_root = images_directory, random_shuffle = True, initial_fill = 21)
        self.decode = ops.ImageDecoder(device = "mixed", output_type = types.RGB)
        self.rotate = ops.Rotate(device = "gpu")
        self.rng = ops.random.Uniform(range = (-10.0, 10.0))
        self.coin = ops.random.CoinFlip(probability = 0.5)
        self.res = ops.Resize(device="gpu", resize_x=224, resize_y=224, interp_type=types.INTERP_TRIANGULAR)
        self.flip = ops.Flip(device = "gpu")

    def define_graph(self):
        jpegs, labels = self.input()
        images = self.decode(jpegs)
        angle = self.rng()
        images = self.rotate(images, angle=angle)
        images = self.flip(images, horizontal = self.coin(), vertical = self.coin())
        images = self.res(images)
        # images are on the GPU
        return (images, labels)

In [11]:
from nvidia.dali.plugin.pytorch import DALIClassificationIterator

# pipe = ExternalSourcePipeline(data_iterator=iterator, batch_size=16, num_threads=2, device_id=0)
# pipe.build()

pipe = HybridPipeline(batch_size=16, num_threads=2, device_id=0, images_directory=images_directory)
pipe.build()

# first parameter is list of pipelines to run
# second pipeline is output_map that maps consecutive outputs to corresponding names
dali_iter = DALIClassificationIterator([pipe], size=409)

pipe2 = HybridPipeline(batch_size=2, num_threads=2, device_id=0, images_directory="/content/flower_data/valid/")
pipe2.build()

valid_iter = DALIClassificationIterator([pipe2], size=409)

/usr/local/lib/python3.7/dist-packages/nvidia/dali/plugin/base_iterator.py:157: Warning: Please set `reader_name` and don't set last_batch_padded and size manually whenever possible. This may lead, in some situations, to missing some samples or returning duplicated ones. Check the Sharding section of the documentation for more details.
  _iterator_deprecation_warning()


In [12]:
import torch
import torch.nn as nn
import torchvision
import torch.optim as optim

if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")

model = torchvision.models.resnet18(pretrained=True)
for name, param in model.named_parameters():
    param.requires_grad = True

model.fc = torch.nn.Sequential(
                torch.nn.Linear(model.fc.in_features, 500),
                torch.nn.ReLU(),
                torch.nn.Dropout(),
                torch.nn.Linear(500, 102),
            )
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [13]:
for epoch in range(20):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, it in enumerate(dali_iter):
        batch_data = it[0]
        inputs, labels = batch_data["data"].to(device, dtype=torch.float32), batch_data["label"].to(device, dtype=torch.int64).squeeze()
        inputs = inputs.permute(0,3,1,2)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()

    
    dali_iter.reset()

    test_correct = 0
    test_total = 0

    with torch.no_grad():
        for i, it in enumerate(valid_iter):
            batch_data = it[0]
            inputs, labels = batch_data["data"].to(device), batch_data["label"].to(device)
            inputs = inputs.permute(0,3,1,2).float()
            labels = labels.long()

            test_output = model(inputs)

            _, test_predicted = torch.max(test_output.data, 1)

            test_total += labels.size(0)
            test_ccount = (test_predicted == labels).sum().item()
            test_correct += test_ccount
    valid_iter.reset()
    print(f"[{epoch}] :  Running Loss = {running_loss}, Correct = {test_correct}, total = {test_total}")

print('Finished Training')

[0] :  Running Loss = 118.76936984062195, Correct = 18, total = 410
[1] :  Running Loss = 108.94468307495117, Correct = 27, total = 408
[2] :  Running Loss = 101.47938251495361, Correct = 30, total = 410
[3] :  Running Loss = 100.3052875995636, Correct = 32, total = 408
[4] :  Running Loss = 99.65138673782349, Correct = 24, total = 410
[5] :  Running Loss = 96.86003232002258, Correct = 35, total = 408
[6] :  Running Loss = 90.18352341651917, Correct = 28, total = 410
[7] :  Running Loss = 89.69748067855835, Correct = 25, total = 408
[8] :  Running Loss = 93.55246686935425, Correct = 35, total = 410
[9] :  Running Loss = 84.10302639007568, Correct = 32, total = 408
[10] :  Running Loss = 87.97801351547241, Correct = 33, total = 410
[11] :  Running Loss = 83.00080513954163, Correct = 35, total = 408
[12] :  Running Loss = 83.99591112136841, Correct = 29, total = 410
[13] :  Running Loss = 78.60520029067993, Correct = 33, total = 408
[14] :  Running Loss = 78.66701817512512, Correct = 41,

In [14]:
!find "/content/flower_data/train/" -type f -print | wc -l
!find "/content/flower_data/valid/" -type f -print | wc -l

6552
818
